In [320]:
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from collections import defaultdict
from statistics import mean
from sentence_transformers import SentenceTransformer
from scipy.spatial import distance
import matplotlib.pyplot 

In [321]:
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]
    
#uso il tokenize_sentence perche poi mi servira' in futuro per la gestione delle frasi, con la relativa ricerca della parola piu frequente in una frase
def tokenize_sentences(sentence):
    word_list= []
    lemma = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if(tag[1][:2] == 'NN'):
            word_list.append(lemma.lemmatize(tag[0], pos = wn.NOUN))
        elif(tag[1][:2] == 'VB'):
            word_list.append(lemma.lemmatize(tag[0], pos = wn.VERB))
        elif(tag[1][:2] == 'RB'):
            word_list.append(lemma.lemmatize(tag[0], pos = wn.ADV))
        elif(tag[1][:2] == 'JJ'):
            word_list.append(lemma.lemmatize(tag[0], pos = wn.ADJ))
    
    return word_list

def pre_processing (sentence):
    return remove_stopwords(tokenize_sentences(remove_punctuation(sentence)))

In [322]:
#in questa sezione sto dicendo che: data una parola in input, in questo caso Courage, la sua definizione ha lunghezza 1, e e in wordnet abbiamo solo 1 concetto associato
def defintion_lenght(word):
    lens = {}

    for sys in wn.synsets(word):
        sys_lenght=[]
        l = len(sys.definition().split())
        if lens.keys().__contains__(l):
            lens[l] += 1
        else:
            lens[l]= 1
    
    ordered = dict(sorted(lens.items(), key=lambda item: item[0]))

    print(ordered)
    

    #provo a plottare dei risultati 
    # using the variable ax for single a Axes
    '''
    fig, ax = matplotlib.pyplot.subplots()
    ax.plot(ordered.keys(), ordered.values(), label='linear')
    ax.set_xlabel('Lunghezza della definizione')
    ax.set_ylabel('Count')
    matplotlib.pyplot.show()
    '''
    #return lens

In [323]:
for w in ['Courage', 'Paper', 'Apprehension', 'Sharpener']: 
    print("\n------------------------\n")
    print("Concept: ", w)
    defintion_lenght(w)


------------------------

Concept:  Courage
{15: 1}

------------------------

Concept:  Paper
{3: 2, 5: 1, 6: 1, 8: 1, 11: 2, 14: 1, 15: 1}

------------------------

Concept:  Apprehension
{2: 1, 4: 1, 7: 1, 8: 1}

------------------------

Concept:  Sharpener
{14: 1}


In [324]:
def hyper_antonym_mero(word):
    hyper_count = 0
    mero_count = 0
    antonym_count = 0

    for sys in wn.synsets(word):
        definition = sys.definition().split()
        hypernym = set(sys.hypernyms())
        antonyms = []

        for l in sys.lemmas():
            antonyms += l.antonyms()
        antonyms = list(map(lambda x : x.synset(), antonyms))
        meronyms =sys.part_meronyms()
        meronyms += sys.substance_meronyms()

        senses = []

        for w in definition:
            senses += wn.synsets(w)
        if (set(senses).intersection(hypernym)):
            hyper_count+=1
        if(set(senses).intersection(antonyms)):
            antonym_count+=1
        if(set(senses).intersection(meronyms)):
            mero_count+=1       
    
    print("\ntot definizioni :" + str(len(wn.synsets(word))) + "\tiperonimi trovati : " + str(hyper_count) + "\t==> " +str(hyper_count/len(wn.synsets(word))))
    print("tot definizioni : " + str(len(wn.synsets(word))) + "\tmeronimi trovati : " + str(mero_count) + "\t==> " +str(mero_count/len(wn.synsets(word))))
    print("tot definizioni : " + str(len(wn.synsets(word))) + "\tantonimi trovati : " + str(antonym_count) + "\t==> = " +str(antonym_count/len(wn.synsets(word))))



In [325]:
for w in ['Courage', 'Paper', 'Apprehension', 'Sharpener']: 
    print("\n------------------------\n")
    print("Concept: ", w)
    hyper_antonym_mero(w)


------------------------

Concept:  Courage

tot definizioni :1	iperonimi trovati : 1	==> 1.0
tot definizioni : 1	meronimi trovati : 0	==> 0.0
tot definizioni : 1	antonimi trovati : 0	==> = 0.0

------------------------

Concept:  Paper

tot definizioni :9	iperonimi trovati : 7	==> 0.7777777777777778
tot definizioni : 9	meronimi trovati : 1	==> 0.1111111111111111
tot definizioni : 9	antonimi trovati : 0	==> = 0.0

------------------------

Concept:  Apprehension

tot definizioni :4	iperonimi trovati : 1	==> 0.25
tot definizioni : 4	meronimi trovati : 0	==> 0.0
tot definizioni : 4	antonimi trovati : 0	==> = 0.0

------------------------

Concept:  Sharpener

tot definizioni :1	iperonimi trovati : 1	==> 1.0
tot definizioni : 1	meronimi trovati : 0	==> 0.0
tot definizioni : 1	antonimi trovati : 0	==> = 0.0


In [326]:
def total_hypernym_path(word):
    lens = []

    for sys in wn.synsets(word):
        path=[]

        hyp_path = sys.hypernym_paths()

        for i in range(0, len(hyp_path[0])):
            path.append((hyp_path[0][i], len((hyp_path[0][i].definition()).split())))
            #path.append(hyp_path[0][i])

        print(path)
        lens.append(path)

    return lens

In [327]:
for w in ['Courage', 'Paper', 'Apprehension', 'Sharpener']: 
    print("\n------------------------\n")
    print("Concept: ", w)
    total_hypernym_path(w)


------------------------

Concept:  Courage
[(Synset('entity.n.01'), 17), (Synset('abstraction.n.06'), 11), (Synset('attribute.n.02'), 9), (Synset('trait.n.01'), 7), (Synset('character.n.03'), 18), (Synset('spirit.n.03'), 9), (Synset('courage.n.01'), 15)]

------------------------

Concept:  Paper
[(Synset('entity.n.01'), 17), (Synset('physical_entity.n.01'), 6), (Synset('matter.n.03'), 7), (Synset('substance.n.01'), 11), (Synset('material.n.01'), 12), (Synset('paper.n.01'), 15)]
[(Synset('entity.n.01'), 17), (Synset('abstraction.n.06'), 11), (Synset('communication.n.02'), 12), (Synset('written_communication.n.01'), 10), (Synset('writing.n.02'), 24), (Synset('essay.n.01'), 6), (Synset('composition.n.08'), 8)]
[(Synset('entity.n.01'), 17), (Synset('physical_entity.n.01'), 6), (Synset('object.n.01'), 12), (Synset('whole.n.02'), 11), (Synset('artifact.n.01'), 7), (Synset('instrumentality.n.03'), 13), (Synset('medium.n.01'), 9), (Synset('print_media.n.01'), 6), (Synset('press.n.02'), 16),